# Sentiment Analysis using Pytorch and Torchtext

- bidirectional LSTM on some news
- some labeled data on news....

- Torchtext to help us numericalize and load some data
- Torchtext is backed by PyTorch....so naturally torchtext is quite good
- Torchtext is not meant to replace spacy....spacy is still like better in general
  
- PyTorch to help us make some neural network